In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

In [36]:
!pip install imbalanced-learn==0.12.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 6.0 MB/s eta 0:00:00:00:01
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.13.0
    Uninstalling imbalanced-learn-0.13.0:
      Successfully uninstalled imbalanced-learn-0.13.0


In [5]:
np.random.seed(123)
torch.manual_seed(123)

In [6]:
df = pd.read_csv('../input/eeg-psychiatric-disorders-dataset/EEG.machinelearing_data_BRMH.csv')
df.shape

(945, 1149)

In [7]:
columns_COH = [col for col in df if col.startswith('COH')]
df_fc = df[columns_COH]
df_fc

,COH.A.delta.a.FP1.b.FP2,COH.A.delta.a.FP1.c.F7,COH.A.delta.a.FP1.d.F3,COH.A.delta.a.FP1.e.Fz,COH.A.delta.a.FP1.f.F4,COH.A.delta.a.FP1.g.F8,COH.A.delta.a.FP1.h.T3,COH.A.delta.a.FP1.i.C3,COH.A.delta.a.FP1.j.Cz,COH.A.delta.a.FP1.k.C4,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
0,49.346741,40.235120,56.181835,49.858714,28.297071,18.089521,22.598544,28.328151,29.067040,27.911928,...,55.989192,16.739679,23.452271,45.678820,30.167520,16.918761,48.850427,9.422630,34.507082,28.613029
1,76.950968,60.256175,77.374983,72.895431,61.492831,38.103992,41.007326,53.906695,54.375496,37.615250,...,45.595619,17.510824,26.777368,28.201062,57.108861,32.375401,60.351749,13.900981,57.831848,43.463261
2,74.336922,79.309590,73.301047,77.045829,47.553073,29.114105,37.764503,40.502080,34.727810,19.250300,...,99.475453,70.654171,39.131547,69.920996,71.063644,38.534505,69.908764,27.180532,64.803155,31.485799
3,29.631700,41.589723,66.176221,44.577578,60.429280,19.669577,34.926288,24.364206,24.165479,20.759497,...,59.986561,63.822201,36.478254,47.117006,84.658376,24.724096,50.299349,35.319695,79.822944,41.141873
4,66.392189,41.839478,64.619452,65.225754,47.783486,31.993546,21.655290,43.369933,46.444416,19.057848,...,61.462720,59.166097,51.465531,58.635415,80.685608,62.138436,75.888749,61.003944,87.455509,70.531662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,60.808135,26.753708,40.633784,35.610775,29.464981,19.457772,18.285060,16.273778,18.295065,17.227339,...,82.905657,34.850706,63.970519,63.982003,51.244725,62.203684,62.062237,31.013031,31.183413,98.325230
941,83.963210,62.958227,82.710283,84.036930,83.985039,73.646169,51.843755,72.904678,84.047671,73.611885,...,65.917918,66.700117,44.756285,49.787513,98.905995,54.021304,93.902401,52.740396,92.807331,56.320868
942,99.526660,99.532797,99.677393,99.704916,76.288907,32.532533,40.445783,66.086634,61.121397,44.056731,...,61.040959,27.632209,45.552852,33.638817,46.690983,19.382928,41.050717,7.045821,41.962451,19.092111
943,99.385867,70.329473,87.395875,54.386213,83.051830,73.013134,63.032383,67.511662,65.910687,53.189822,...,99.113664,48.328934,41.248470,28.192238,48.665743,42.007147,28.735945,27.176500,27.529522,20.028446


In [34]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['main.disorder'].to_numpy())
y.shape

(945,)

In [38]:
from imblearn.over_sampling import BorderlineSMOTE

borderline_smote = BorderlineSMOTE(k_neighbors=3, random_state=42)
X_resampled, y_resampled = borderline_smote.fit_resample(df_fc, y)
X_resampled.shape, y_resampled.shape

((1862, 1026), (1862,))

In [39]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_resampled = scaler.fit_transform(X_resampled)

In [45]:
n_seq = 6
n_features = 171
rows = X_resampled.shape[0]

In [48]:
X_seq = X_resampled.reshape(rows, n_seq, n_features)
X_seq.shape

(1862, 6, 171)

In [51]:
X_tensor = torch.tensor(X_seq, dtype=torch.float32)
y_tensor = torch.tensor(y_resampled, dtype=torch.long)
X_tensor.shape, y_tensor.shape

(torch.Size([1862, 6, 171]), torch.Size([1862]))

In [52]:
X_train, X_val, y_train, y_val = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

In [57]:
class EEGLSTMClassifier(nn.Module):
    def __init__(self, input_size=171, hidden_size=64, num_classes=7):
        super(EEGLSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # x: (batch, seq_len, input_size)
        _, (h_n, _) = self.lstm(x)  # h_n: (1, batch, hidden_size)
        h_n = h_n.squeeze(0)        # (batch, hidden_size)
        out = self.fc(h_n)          # (batch, num_classes)
        return out

In [58]:
model = EEGLSTMClassifier(input_size=171, hidden_size=258, num_classes=7)

In [59]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [60]:
n_epochs = 100
batch_size = 32

for epoch in range(n_epochs):
    model.train()
    permutation = torch.randperm(X_train.size(0))

    for i in range(0, X_train.size(0), batch_size):
        indices = permutation[i:i+batch_size]
        batch_X, batch_y = X_train[indices], y_train[indices]

        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    # Validação
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_preds = torch.argmax(val_outputs, dim=1)
        val_acc = (val_preds == y_val).float().mean().item()
    print(f"Epoch {epoch+1}, Val Accuracy: {val_acc:.4f}")

Epoch 1, Val Accuracy: 0.3861
Epoch 2, Val Accuracy: 0.4504
Epoch 3, Val Accuracy: 0.4933
Epoch 4, Val Accuracy: 0.6005
Epoch 5, Val Accuracy: 0.6193
Epoch 6, Val Accuracy: 0.6676
Epoch 7, Val Accuracy: 0.6729
Epoch 8, Val Accuracy: 0.6863
Epoch 9, Val Accuracy: 0.6917
Epoch 10, Val Accuracy: 0.7131
Epoch 11, Val Accuracy: 0.7212
Epoch 12, Val Accuracy: 0.7346
Epoch 13, Val Accuracy: 0.7051
Epoch 14, Val Accuracy: 0.7185
Epoch 15, Val Accuracy: 0.7131
Epoch 16, Val Accuracy: 0.7212
Epoch 17, Val Accuracy: 0.7131
Epoch 18, Val Accuracy: 0.7292
Epoch 19, Val Accuracy: 0.7292
Epoch 20, Val Accuracy: 0.7265
Epoch 21, Val Accuracy: 0.7212
Epoch 22, Val Accuracy: 0.7185
Epoch 23, Val Accuracy: 0.7212
Epoch 24, Val Accuracy: 0.7212
Epoch 25, Val Accuracy: 0.7212
Epoch 26, Val Accuracy: 0.7185
Epoch 27, Val Accuracy: 0.7239
Epoch 28, Val Accuracy: 0.7212
Epoch 29, Val Accuracy: 0.7265
Epoch 30, Val Accuracy: 0.7212
Epoch 31, Val Accuracy: 0.7319
Epoch 32, Val Accuracy: 0.6702
Epoch 33, Val Acc